In [133]:
import pandas as pd # pour traiter les tableaux
import shutil # pour gérer les fichiers et en particulier copier le fichier modèle

In [41]:
stat_inscrits = pd.read_excel("statistiques_inscrits.xls", skiprows=3)
stat_inscrits['nom_classe'] = stat_inscrits.Classe.str.split(expand=True).iloc[:,[0]]
stat_inscrits['Niveau'] = stat_inscrits.Classe.str.split(expand=True).iloc[:,[1]]

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [130]:
for nom_classe in ['Aiga', "Tèrra", "Fuòc", "Amor", "Aire"]:
    Classe = stat_inscrits[stat_inscrits.nom_classe==nom_classe]
    Classe = Classe[Classe.Groupe.isin(['CLAE Matin', 'CLAE Midi', 'CLAE Soir'])]
    Classe = Classe[~Classe.Groupe.isna()]
    Classe = pd.concat([Classe, pd.get_dummies(Classe.Clae)[['Matin', 'Midi', 'Soir']]], axis=1)
    Classe_pivot = Classe.pivot_table(values=['Matin', 'Midi', 'Soir'], index=['Enfant', 'Niveau'], columns=['Début inscr.'], aggfunc='sum')#.swaplevel(0, 1, axis=1)
    Classe_pivot = Classe_pivot.reorder_levels([1, 0], axis=1).sort_values('Début inscr.', axis=1)
    with pd.ExcelWriter("Suivi Des Presences Hebdomadaire-Modele.xlsx", mode="a", engine="openpyxl", if_sheet_exists="replace") as writer:
        Classe_pivot.to_excel(writer, sheet_name=nom_classe, startrow=4)  